In [311]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer

In [87]:
twitter=pd.read_csv(r"0000000000002747_training_twitter_x_y_train .csv")
twitter2=pd.read_csv(r"0000000000002747_test_twitter_x_test.csv")

In [88]:
y_train,x_train=twitter.airline_sentiment,twitter.text
x_test=twitter2.text
type(x_train[0])

str

In [89]:
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [90]:
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)

#stops, string.punctuation

In [91]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [92]:
x_train=[word_tokenize(x) for x in x_train]

In [93]:
x_train = [clean_review(x) for x in x_train]

In [94]:
x_train=[" ".join(x) for x in x_train]

In [64]:
x_train[0]

'southwestair schedule morning 2 day fact yes..not sure even flight one cancelled flightled'

In [320]:
count_vec=CountVectorizer(max_features = 2000, ngram_range=(1,2), max_df=.60, min_df=1)

In [308]:
x_train_features = count_vec.fit_transform(x_train)
x_train_features.todense()

matrix([[1, 0],
        [0, 0],
        [0, 1],
        ...,
        [0, 0],
        [0, 0],
        [1, 1]], dtype=int64)

In [407]:
tf = TfidfVectorizer(ngram_range=(1, 2), max_df=0.80, min_df=2)
txt_fitted = tf.fit(x_train)
txt_transformed = txt_fitted.transform(x_train)
#txt_transformed.toarray()

In [400]:
#count_vec.get_feature_names()

In [248]:
#from sklearn.svm import SVC
#svc = SVC(C)
#svc.fit(x_train_features, y_train)
#svc.score(x_train_features, y_train)
#y_test=svc.predict(x_test_features)

In [408]:
x_test_features = txt_fitted.transform(x_test)

In [402]:
from sklearn.ensemble import RandomForestClassifier

In [403]:
rf=RandomForestClassifier()
rf.fit(txt_transformed, y_train)
rf.score(txt_transformed, y_train)

0.9746812386156648

In [393]:
y_test=rf.predict(x_test_features)

In [409]:
from sklearn.linear_model import LogisticRegression
lreg = LogisticRegression()
lreg.fit(txt_transformed, y_train)
lreg.score(txt_transformed, y_train)

0.8410746812386156

In [410]:
y_test=lreg.predict(x_test_features)

In [420]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
clf = SVC(gamma=0.005,C=100.0)
#grid = {'C' : [1, 10 , 1e2, 1e3, 5e3, 1e4, 5e4, 1e5],
       #'gamma' : [1e-3, 5e-4, 1e-4, 5e-3]}
#abc = GridSearchCV(clf, grid)
clf.fit(txt_transformed, y_train)
#abc.best_estimator_

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.005, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [421]:
clf.score(txt_transformed, y_train)

0.918943533697632

In [422]:
np.savetxt('predtfifd7.csv', y_test, fmt="%s", delimiter=",")